# EE5907 Programming Assignment Q3
Lee Jianwei
A0018867

In [4]:
from __future__ import division
import scipy.io
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Pool
np.set_printoptions(precision=2, suppress=True)

# Load Data

In [5]:
d = scipy.io.loadmat('spamData.mat')
ytest = d['ytest'].flatten()
ytrain = d['ytrain'].flatten()
xtest = d['Xtest']
xtrain = d['Xtrain']

In [6]:
def binarize(array):
    """array: list of emails, each 57 features long"""
    return np.array([x>0 for x in array]).astype('uint8')
xtrainBin = binarize(xtrain)
xtestBin = binarize(xtest)

In [11]:
#z-normalise features
def znorm1D(array1D):
    m = np.mean(array1D)
    s = np.std(array1D)
    return np.array((array1D-m)/s)
def znorm2D(array2D):
    """znorm along columns of 2D array"""
    znormed_columns = np.array([znorm1D(c) for c in array2D.T]) # each column in array2D is a row (called c) in array2D.T
    return znormed_columns.T
xtrainZ = znorm2D(xtrain)
xtestZ = znorm2D(xtest)

In [13]:
#log-transform features
def log2D(array):
    """array: list of emails, each 57 features long"""
    return np.array([np.log(x+0.1) for x in array]).astype('uint8') #x is a 57 element array
xtrainLog = log2D(xtrain)
xtestLog = log2D(xtest)

In [7]:
classMLE = np.average(ytrain)

In [1]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [8]:
np.dot(xtrainBin,np.ones(57))

array([  8.,   4.,  33., ...,  30.,   7.,  16.])

In [11]:
np.sum(xtrainBin[1])

4

In [18]:
ytrain

array([1, 0, 1, ..., 1, 1, 0], dtype=uint8)

In [160]:
def reg_params(x_array2D, y_array1D, w, lamb):
    """returns NLL, g, H"""
    N_features = x_array2D.shape[1] # number of features per mail: default is 57
    x_array2D = np.insert(x_array2D,0,1,axis=1) # modify feature list to include a 1 at the start
    mu = sigmoid(-np.dot(x_array2D, w)) # array1D of mus, one for each email
    NLL = -np.sum([y*np.log(m) + (1-y)*np.log(1-m) for y,m in zip (y_array1D, mu)])
    g = np.dot(x_array2D.T,(mu-y_array1D))
    S = np.diag(mu*(1-mu))
    H = np.dot(x_array2D.T, np.dot(S, x_array2D))
    
    NLL_reg = NLL + 0.5*lamb*np.inner(w[1:],w[1:])
    g_reg = g + np.insert(lamb*w[1:],0,0) # first element of g is unregularised
    H_reg = H + np.diag(np.insert(lamb*np.ones(N_features),0,0)) # first diagonal element is unregularised
    return NLL_reg, g_reg, H_reg


In [159]:
NLL_reg, g_reg, H_reg = reg_params(xtrainZ,ytrain,np.ones(58),1)

[ 542.5  -473.83   92.66 -580.71 -144.76 -715.53 -594.98 -760.39 -602.75
 -581.66 -493.48 -712.91 -208.98 -409.25 -216.97 -594.77 -631.88 -694.28
 -586.9  -647.15 -587.88 -957.1  -299.51 -878.86 -518.09  162.79   72.63
  440.43  -13.37   -8.88  -18.94  -54.56  -65.23  147.44  -69.2     2.87
  -17.29   37.88  -26.6     1.35 -207.82   17.65  112.11  -62.5    82.32
  165.74  208.53  -35.78   34.15  -38.33  -27.    -45.42 -500.6  -816.7
 -227.96 -304.52 -536.13 -673.05] [ 542.5  -472.83   93.66 -579.71 -143.76 -714.53 -593.98 -759.39 -601.75
 -580.66 -492.48 -711.91 -207.98 -408.25 -215.97 -593.77 -630.88 -693.28
 -585.9  -646.15 -586.88 -956.1  -298.51 -877.86 -517.09  163.79   73.63
  441.43  -12.37   -7.88  -17.94  -53.56  -64.23  148.44  -68.2     3.87
  -16.29   38.88  -25.6     2.35 -206.82   18.65  113.11  -61.5    83.32
  166.74  209.53  -34.78   35.15  -37.33  -26.    -44.42 -499.6  -815.7
 -226.96 -303.52 -535.13 -672.05]


In [134]:
np.insert(xtrainBin,0,1,axis=1)

array([[1, 0, 0, ..., 1, 1, 1],
       [1, 0, 0, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ..., 
       [1, 0, 1, ..., 1, 1, 1],
       [1, 0, 0, ..., 1, 1, 1],
       [1, 0, 0, ..., 1, 1, 1]], dtype=uint8)

In [154]:
np.diag(np.insert(np.ones(57),0,0))

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])